## 기타(이미지 분석 gradio 예제)

In [1]:
import ollama
import gradio as gr
import base64
from PIL import Image
import io

c:\pythonEdu\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def encode_img(img_path):
    """Encode image to base64 string"""
    with open(img_path, 'rb') as img_file:  # rb: 바이너리 읽기(이미지는 0,1 형태의 바이너리)
        return base64.b64encode(img_file.read()).decode('utf-8')
    

In [4]:
def analyze_image_with_gemma(img):
    """Ollama Gemma3 4B model을 사용하여 이미지 분석"""
    try:
        # Convert Gradio image to PIL Image
        pil_img = Image.fromarray(img)    # PIL(이미지처리, 조작) 이미지 객체로 변환
        
        # Save image to a temporary buffer
        buff = io.BytesIO()    # 메모리 상에 임시 버퍼를 만들어서 이미지 데이터 저장
        pil_img.save(buff,format='PNG')
        
        # Encode image to base64
        encode_img = base64.b64encode(buff.getvalue()).decode('utf-8')    # 임시 버퍼에 저장된 이미지 데이터를 바이트 문자열로 가져옴
        
        # Perform image analysis using Ollama Gemma3
        response = ollama.chat(
            model = 'gemma3:4b',
            messages=[
                {
                    'role': 'user',
                    'content': '이미지의 내용을 자세히 설명해줘. 무엇이 보이는지, 색상, 구성, 감정 등을 포함해서 분석해줘.',
                    'images': [encode_img]
                }
            ]
        )
        
        return response['message']['content']
    except Exception as e:
        return f"이미지 분석 중 오류 발생: {str(e)}"

In [ ]:
iface = gr.Interface(
    fn = analyze_image_with_gemma,
    inputs = gr.Image(type='numpy', label="이미지 업로드"),
    outputs = gr.Textbox(label="이미지 분석결과"),
    title = "Ollama Gemma3 이미지 분석기",
    description="이미지를 업로드하면 Gemma3 4B 모델이 분석해드립니다."
)

In [6]:
iface.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Created dataset file at: .gradio\flagged\dataset2.csv


In [7]:
iface.close()

Closing server running on port: 7860


## 챗봇 예제(역할 부여)

In [14]:
# 사전 설치 : pip install ollama
import ollama

In [15]:
# 챗봇의 기본적 질문, 답변 역할
def ask_ollama(question):
    # ollama를 사용하여 모델로부터 응답 생성
    cr = "너는 의사야. 질문에 대한 답은 3줄 이내로 짧게해줘."
    res = ollama.chat(model="gemma2", messages=[
        {'role':'system', 'content': cr},    # 챗봇의 기본 역할 부여
        {'role':'user', 'content': question}        # 질문
    ])
    
    return res['message']['content']

In [19]:
q = "3일전에 축구한 이후로 발목이 아파"
res = ask_ollama(q)
print(res)

발목 통증은 부상으로 인한 것이 가능합니다.  

휴식, 얼음찜질, 가벼운 압박을 해주세요. 만약 통증이 심하거나 움직이기 어렵다면 병원에 방문하세요.





## 챗봇 예제(Gradio 사용)

In [20]:
# 사전 설치 : pip install gradio
from langchain_community.chat_models import ChatOllama
from langchain.schema import HumanMessage, AIMessage    # HumanMessage: 사용자가 보낸 메시지, AIMessage : LLM의 메시지
import gradio as gr

In [26]:
# ChatOllama 모델 초기화
model = ChatOllama(model="gemma3:4b", temperature=0.1, verbose=False)    # temperture가 낮을수록 거의 동일답변, 높을수록 창의적인 답변

In [27]:
# 채팅 기록을 포함하여 응답을 생성하는 함수
def chat(msg, his):
    # 이전 대화 기록을 ChatOllama 형식으로 변환
    chat_his = []
    for human, ai in his:
        chat_his.append(HumanMessage(content=human))
        chat_his.append(AIMessage(content=ai))
        
    # 현재 메시지 추가
    chat_his.append(HumanMessage(content=msg))
    
    # 모델을 사용하여 응답 생성
    res = model.invoke(chat_his)
    
    return res.content

In [28]:
# Gradio 인터페이스 설정
demo = gr.ChatInterface(
    fn = chat,
    examples=[
        "안녕하세요!",
        "인공지능에 대해 설명해주세요.",
        "파이썬의 장점은 무엇인가요?"
    ],
    title='AI 챗봇',
    description="질문을 입력하면 AI가 답변합니다."
)

c:\pythonEdu\.venv\lib\site-packages\gradio\chat_interface.py:334: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


In [29]:
# 서버 실행
demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [47]:
demo.close()

Closing server running on port: 7860


## 챗봇 예제(Gradio + csv 사용)

In [39]:
import pandas as pd
from langchain_community.chat_models import ChatOllama
from langchain.schema import HumanMessage, AIMessage   # HumanMessage: 사용자가 보낸 메시지, AIMessage : LLM의 메시지
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter  # 특정 문자(예: 줄바꿈, 공백)를 기준으로 텍스트를 분할하는 기능을 제공
from langchain.chains import ConversationalRetrievalChain  # 질문과 관련된 정보 검색, 이전 대화정보도 같이 LLM에 제공, 답변생성
import gradio as gr

In [40]:
# CSV 파일 로드
df = pd.read_csv("dataset/indata_kor.csv", encoding='CP949')

In [43]:
# 텍스트 분할
txt_split = CharacterTextSplitter(chunk_size=1000,chunk_overlap=200)    # 텍스트 조각 최대 1000자, 텍스트 조각 사이에 200자만큼의 중복을 허용(문맥 유지)
txts = txt_split.split_text('\n'.join(df.to_string()))    # 문자열들을 줄바꿈 문자(\n)를 기준으로 연결

In [45]:
# 임베딩 모델 초기화
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v2")
# 모델 이름 : 조직이름(sentence-transformers) 다양한 작업 가능(all)-MS사 경령화 트랜스포머모델(MiniLM)-모델의 레이어수(L6)-모델이 버전(v2)
emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [46]:
# 벡터 데이터베이스 생성
vector = FAISS.from_texts(txts, emb)    # from_texts : 임베딩으로 변환된 벡터를 FAISS 인덱스에 저장

In [48]:
# ChatOllama 모델 초기화
llm = ChatOllama(model="gemma3:4b", temperature=0.1)    # temperture가 낮을수록 거의 동일답변, 높을수록 창의적인 답변

In [49]:
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    vector.as_retriever(search_kwargs={"k":1}),  # 가장 관련성 높은 1개의 문서만 검색
    return_source_documents=True,    # 참고한 소스 문서 정보 반환 여부
    verbose=False    # 체인의 실행 과정 출력 여부
)    

In [50]:
# 채팅 함수 정의
def chat(msg,his):
    # 이전 대화 기록을 ConversationalRetrievalChain 형식으로 변환
    chat_his = [(human,ai) for human, ai in his]
    
    # 모델을 사용하여 응답 생성
    res = qa_chain({"question": msg, "chat_history":chat_his})
    
    # 소스 문서 정보 추출
    sources = set([doc.metadata.get('source','Unknown') for doc in res['source_documents']])
    source_info = f"\n\n참고 출처: {', '.join(sources)}" if sources else ""
    
    return res['answer'] + source_info

In [52]:
# Gradio 인터페이스 설정
demo = gr.ChatInterface(
    fn = chat,
    examples=[
        "한국폴리텍대학 스마트금융과 면접시에는 어떤걸 준비하고 가면 될까요?",
        "스마트금융과에 대해 설명해주세요",
        "한국폴리텍대한 추천할만한 학과 하나를 소개해주세요."
    ],
    title = "대학 정보 AI 챗봇",
    description="스마크금융과에 대한 질문을 입력하면 AI가 CSV데이터를 참고하여 한글로 답변합니다."
)

c:\pythonEdu\.venv\lib\site-packages\gradio\chat_interface.py:334: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


In [53]:
# 서버 실행
demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


C:\Users\human-16\AppData\Local\Temp\ipykernel_3612\2545116048.py:7: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res = qa_chain({"question": msg, "chat_history":chat_his})


In [64]:
demo.close()

Closing server running on port: 7860


## 챗봇 예제(인터넷 URL정보 요약하기 + TAB추가)

In [ ]:
# Function to load, split, and retrieve documents
    # vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

In [54]:
# Function to format documents

In [ ]:
# Function that defines the RAG chain

In [ ]:
# Gradio interface

# iface = gr.Interface(
#     fn=rag_chain,
#     inputs=["text", "text"],
#     outputs="text",
#     title="RAG Chain Question Answering",
#     description="Enter a URL and a query to get answers from the RAG chain."
# )

In [ ]:
# Gradio Tabbed Interface
    # Tab for Question and Answer
    # Tab for Visualization (Word Cloud)

In [55]:
# 디버그 모드로 Gradio 인터페이스 실행

## 챗봇 예제(STT:음성을 텍스트로 전환)

In [9]:
# 사전 설치 : pip install openai-whisper
# ffmpeg 사전 설치 및 환경변수 path 설정(경로/bin)
import os
from dotenv import load_dotenv  # 환경변수 로드가 필요한 경우
import whisper
import gradio as gr

In [10]:
# .env 파일에서 환경 변수 로드 (필요한 경우)
# load_dotenv()

In [11]:
# ffmpeg 경로 명시적 설정
# os.environ["FFMPEG_BINARY"] = "C:/aiproject/ffmpeg/bin/ffmpeg.exe"
os.environ["PATH"] += os.pathsep + r"C:\ffmpeg\bin"
os.environ["FFMPEG_BINARY"] = r"C:\ffmpeg\bin\ffmpeg.exe"

In [12]:
def transcribe_audio(audio_path):
    # Whisper 모델 로드
    model = whisper.load_model("base")

    # 오디오 파일 전사
    result = model.transcribe(audio_path)

    # 전사된 텍스트 반환
    return result["text"]

In [13]:
def process_audio(audio):
    if audio is None:
        return "오디오 파일을 업로드해주세요."
    try:
        transcribed_text = transcribe_audio(audio)
        return transcribed_text
    except Exception as e:
        return f"오류가 발생했습니다: {str(e)}"

In [14]:
# Gradio 인터페이스 생성
iface = gr.Interface(
    fn=process_audio,
    inputs=gr.Audio(type="filepath", label="MP3 파일 업로드"),
    outputs="text",
    title = "MP3 to Text Converter",
    description="MP3 파일을 업로드하면 텍스트로 변환합니다."
)

In [15]:
# 디버그 모드로 Gradio 인터페이스 실행
iface.launch(server_port=7861, server_name="0.0.0.0", debug=True)

* Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


100%|███████████████████████████████████████| 139M/139M [00:13<00:00, 11.1MiB/s]
c:\pythonEdu\.venv\lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Keyboard interruption in main thread... closing server.


In [ ]:
iface.close()

Closing server running on port: 7860


## 챗봇 예제(TTS:텍스트를 음성변환)

In [2]:
# 사전 설치 : pip install gtts
import gradio as gr
from gtts import gTTS
import os
import tempfile

In [3]:
def txt_to_spch(txt, lang='ko')    :
    # 임시 파일 생성
    with tempfile.NamedTemporaryFile(delete=False,suffix=".mp3") as fp:
        temp_filename = fp.name
    # TTS 변환
    tts = gTTS(text=txt,lang=lang)
    tts.save(temp_filename)
    return temp_filename

In [4]:
def process_tts(txt, lang):
    if not txt:
        return None, "텍스트를 입력해주세요."
    try:
        audio_file = txt_to_spch(txt, lang)
        return audio_file, "변환이 완료되었습니다. 아래에서 재생또는 다운로드할 수있습니다."
    except Exception as e:
        return None, f"오류발생: {str(e)}"

In [5]:
# Gradion 인터페이스 생성
iface = gr.Interface(
    fn = process_tts,
    inputs=[
        gr.Textbox(lines=5, label="텍스트 입력"),
        gr.Dropdown(choices=['ko','en','ja','zh-cn'], label='언어 선택', value='ko')
    ],
    outputs=[
        gr.Audio(label="생성된 오디오"),
        gr.Textbox(label="상태 메시지")
    ],
    title="Text to Speech Converter",
    description="텍스트를 입력하면 MP3 파일로 변환합니다."
)

In [6]:
# 디버그 모드로 Gradio 인터페이스 실행
iface.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [7]:
iface.close()

Closing server running on port: 7860


## 이미지 분류 예제(Gradio 사용)

In [69]:
# 사전설치 : pip install pillow

In [70]:
# TensorFlow MobileNetV2 모델 로드

In [71]:
        # URL에서 이미지 가져오기
            # BytesIO 사용하여 이미지 열기
        # 이미지를 배열로 변환
            # 배치 차원 추가
            # 전처리
        # 예측 수행
            # 상위 3개 예측 결과 반환
        # Gradio Label 컴포넌트에 맞게 결과 형식 변경
        # {label1: confidence1, label2: confidence2, ...} 형식으로 반환
            # 에러 발생 시 기본값 반환

In [72]:
# Gradio 인터페이스 생성

In [74]:
# 인터페이스 실행
# 예시 이미지 URL : https://health.chosun.com/site/data/img_dir/2024/04/19/2024041901914_0.jpg

## 이미지 분류 예제(Gradio + gemma2 사용)

In [75]:
# TensorFlow MobileNetV2 모델 로드

In [76]:
    # 로컬 서버 주소
    # 사용하려는 Ollama 모델 이름

In [77]:
# Ollama를 사용해 음식 설명 생성

In [78]:
# 이미지 예측 함수
        # URL에서 이미지 가져오기
            # BytesIO 사용하여 이미지 열기
        # 이미지를 배열로 변환
            # 배치 차원 추가
            # 전처리
        # 예측 수행
            # 상위 3개 예측 결과 반환
        # 예측 결과 형식화
        # 가장 높은 확률의 예측값으로 Ollama 설명 생성
            # 가장 확률이 높은 음식 이름
            # 예측 결과와 Ollama 설명 반환
            # 에러 발생 시 기본값 반환

In [79]:
# Gradio 인터페이스 생성
        # 상위 3개 예측 결과
        # Ollama로 생성한 설명 출력

In [80]:
# 인터페이스 실행